In [0]:
df=spark.read.format("csv").option("Inferchema","True").option("Header","True").load("/Volumes/medallion_architecture/demo/demo/sales.csv")

In [0]:
df.write.format("delta").mode("overwrite").save(
    "/Volumes/medallion_architecture/demo/demo/bronze/sales"
)

In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark, "/Volumes/medallion_architecture/demo/demo/bronze/sales")
df_silver=deltaTable.toDF()

In [0]:
df_silver.display()

In [0]:
df_silver.write.format("delta").mode("overwrite").save("/Volumes/medallion_architecture/demo/demo/silver/sales")

In [0]:
from delta.tables import DeltaTable
deltaTable = DeltaTable.forPath(spark, "/Volumes/medallion_architecture/demo/demo/silver/sales")
df_gold=deltaTable.toDF()
df_gold.display()
df_gold.write.format("delta").mode("overwrite").save("/Volumes/medallion_architecture/demo/demo/gold/sales")

In [0]:
df=spark.read.format("delta").option("VersionAsOf",4).load("/Volumes/practice/practice/processed_files/silver/")

In [0]:
df.display()

In [0]:
%sql
SELECT * FROM delta.`/Volumes/practice/practice/processed_files/silver`

In [0]:
%sql
ALTER TABLE delta.`/Volumes/practice/practice/processed_files/silver`
SET TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true');

In [0]:
df5=spark.read.format("delta").option("readchangedata","True").option("startingVersion",3).option("endingVersion",5).load("/Volumes/practice/practice/processed_files/silver")

In [0]:
df5.display()